In [142]:
import yaml
import json
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

from rules_engine import rules_engine
import utils

pd.options.display.max_columns = 999

In [143]:
with open('config.yaml') as f:
	# use safe_load instead load
	data_map = yaml.safe_load(f)
file_length = data_map['file_length']['value']


In [144]:
use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts', 'stateCode']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
			 'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependencies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
					header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)
small_counties = list(cbsas.countyFips[cbsas.smallCounty=="1"])
#load schemas for LAR and transmittal sheet
#schemas contain valid enumerations, including NA values, for each field in the dataset
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))
!pwd

/Users/roellk/hmda/hmda-test-files/python


In [169]:

file = "v608.txt"
#file = "clean_file_100_rows.txt"
if file[0] == "v":
    file_type = "validity"
elif file[0] == "s":
    file_type = "syntax"
elif file[0] == "q":
    file_type = "quality"
else:
    file_type = "clean"
path = "../edits_files/"+file_type+"/"
if file_type == "clean":
    path = "../edits_files/"

#lar validator checks a dataframe and returns a JSON with generate_error_files
checker = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, cbsa_data=cbsas)#tracts=tracts, 
#                       counties=counties, small_counties=small_counties) 
#load data to checker
ts_df, lar_df = utils.read_data_file(path=path, data_file=file)
checker.load_data_frames(ts_df, lar_df)
for func in dir(checker):
    if func[:1] in ("s", "v", "q") and func[1:4].isdigit()==True:
        #print("applying:", func)
        getattr(checker, func)()
res_df = pd.DataFrame(checker.results)
res_df[(res_df.status=="failed")&(res_df.edit_name==file[:-4])]

/Users/roellk/homebrew/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/roellk/homebrew/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,edit_name,fail_count,fields,row_ids,row_type,status


In [128]:
#get all edit test files in single list
val_path = "../edits_files/validity/"
syn_path = "../edits_files/syntax/"
val_files = [f for f in listdir(val_path) if isfile(join(val_path, f)) and f[0]=="v"] #get list of only files
syn_files = [f for f in listdir(syn_path) if isfile(join(syn_path, f)) and f[0]=="s"]

#check validity test files for error rate
for file in val_files:
    print(file)
    #lar validator checks a dataframe and returns a JSON with generate_error_files
    checker = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, cbsa_data=cbsas)#tracts=tracts, counties=counties)
    #load data to checker
    ts_df, lar_df = utils.read_data_file(path=val_path, data_file=file)
    checker.load_data_frames(ts_df, lar_df)
    for func in dir(checker):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            getattr(checker, func)()
    res_df = pd.DataFrame(checker.results)
    res_df = res_df[(res_df.edit_name==file[:-4])] #set res_df for only the edit in the file name
    if res_df.row_type.iloc[0]=="TS" and res_df.status.iloc[0]=="failed":
        print("file is good")
        
    if res_df.row_type[(res_df.edit_name==file[:-4])].iloc[0] == "LAR" and \
       res_df.fail_count[(res_df.edit_name==file[:-4])].iloc[0] != file_length:
        print("WARNING BOOOOOP", len(res_df.fail_count[res_df.edit_name==file[:-4]]))
    elif res_df.row_type[(res_df.edit_name==file[:-4])].iloc[0] == "LAR" and res_df.fail_count[(res_df.edit_name==file[:-4])].iloc[0] == file_length:
        print("file is good")
    print()

#check syntax test files for error rate
for file in syn_files:
    print(file)
    #lar validator checks a dataframe and returns a JSON with generate_error_files
    checker = rules_engine(lar_schema=lar_schema_df, ts_schema=ts_schema_df, tracts=tracts, counties=counties)
    #load data to checker
    ts_df, lar_df = utils.read_data_file(path=syn_path, data_file=file)
    checker.load_data_frames(ts_df, lar_df)
    for func in dir(checker):
        if func[:1] in ("s", "v") and func[1:4].isdigit()==True:
            getattr(checker, func)()
    res_df = pd.DataFrame(checker.results)
    res_df = res_df[(res_df.edit_name==file[:-4])] #set res_df for only the edit in the file name
    
    if res_df.row_type.iloc[0]=="TS" and res_df.status.iloc[0]=="failed":
        print("file is good")
    if res_df.row_type[(res_df.edit_name==file[:-4])].iloc[0] == "LAR":
        fail_count = res_df.fail_count[(res_df.edit_name==file[:-4])].iloc[0]
    
    if row_type == "LAR" and int(fail_count) != file_length:
        print("WARNING BOOOOOP\n", "*"*100)
    else:
        print("file is good")
    print()

In [6]:
#load CBSA data for geography testing edits
use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
'stateCode', 'tractDecimal']
cbsa_data = pd.read_csv('../dependencies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
header=None, names=cbsa_cols, dtype=object) #load tract to CBSA data from platform file


In [133]:
len(lar_df[lar_df.lei.iloc[0]==lar_df.uli.apply(lambda x: x[:20])])

27

In [160]:
row_ids = list(res_df.row_ids[(res_df.status=="failed")&(res_df.edit_name==file[:-4])])
#for row_id in row_ids[0]:
#    print(row_id, len(row_id), row_id[:20]==lar_df.lei.iloc[0])

In [163]:
row_ids = list(res_df.row_ids[(res_df.status=="failed")&(res_df.edit_name==file[:-4])])
for index, row in lar_df[~lar_df.uli.isin(row_ids[0])].iterrows():
#    print(row.uli, len(row.uli), type(row.uli), row.uli[:20]==ts_df.lei.iloc[0])
    print(row.uli, len(row.uli))

W5BV6YCT7D06B9RXF1O2B 21
PI4DRJAWVZFPU98U02EYK 21
PXN13ZNVDL9TIJWR1UWXZ 21
GT7H92U568QVXXR9I03XJ 21
KLAY82I0HSO1S28EOIK1Y 21
RB2GHRGX76G2BY0SP9RWH 21
C6DDWX48AG7OACRYOM5JE 21
AAMG10VXCBB0Y9YRIMTFG 21
3P639HSUNEULFD42X0FPF 21
WSWVMLH20Z3U28PDWA4UA 21
Q0OII2SQ4LBIKTN0UAJ17 21
3Z6RZVTJI0V95P3NSBY2Y 21
XHCD4WHNVT1KISU6KJ3WN 21
LQE90OUUIMGWQNK8KRZII 21
PM5E791784G8IYSIGKIDB 21
GXOFOAYTESSXFHB4BEYR3 21


In [150]:

print(len(lar_df[lar_df.uli.apply(lambda x: x[:20]==ts_df.lei.iloc[0])]))
print(len(lar_df[lar_df.uli.apply(lambda x: x[:20]!=ts_df.lei.iloc[0])]))

27
73


In [151]:
print(len(lar_df[~lar_df.uli.isin(row_ids[0])]))
lar_df[~lar_df.uli.isin(row_ids[0])]

16


field,record_id,lei,uli,app_date,loan_type,loan_purpose,preapproval,const_method,occ_type,loan_amount,action_taken,action_date,street_address,city,state,zip_code,county,tract,app_eth_1,app_eth_2,app_eth_3,app_eth_4,app_eth_5,app_eth_free,co_app_eth_1,co_app_eth_2,co_app_eth_3,co_app_eth_4,co_app_eth_5,co_app_eth_free,app_eth_basis,co_app_eth_basis,app_race_1,app_race_2,app_race_3,app_race_4,app_race_5,app_race_native_text,app_race_asian_text,app_race_islander_text,co_app_race_1,co_app_race_2,co_app_race_3,co_app_race_4,co_app_race_5,co_app_race_native_text,co_app_race_asian_text,co_app_race_islander_text,app_race_basis,co_app_race_basis,app_sex,co_app_sex,app_sex_basis,co_app_sex_basis,app_age,co_app_age,income,purchaser_type,rate_spread,hoepa,lien,app_credit_score,co_app_credit_score,app_score_name,app_score_code_8,co_app_score_name,co_app_score_code_8,denial_1,denial_2,denial_3,denial_4,denial_code_9,loan_costs,points_fees,origination_fee,discount_points,lender_credits,interest_rate,prepayment_penalty,dti,cltv,loan_term,intro_rate,balloon,int_only_pmts,neg_amort,non_amort_features,property_value,manufactured_type,manufactured_interest,total_units,affordable_units,app_submission,initially_payable,mlo_id,aus_1,aus_2,aus_3,aus_4,aus_5,aus_code_5,aus_result_1,aus_result_2,aus_result_3,aus_result_4,aus_result_5,aus_code_16,reverse_mortgage,open_end_credit,business_purpose,fail_flag
4,2,08ZQTJA357VRO2G0ZXGI,KLB6E4CQIUDQG379R81U4S,20180811,2,5,2,2,3,260765,2,20181204,1234 Hocus Potato Way,Tatertown,ID,37375,51041,51041100403,3,,,,,OCLG7DKYHGRFJLFFO5TR079U3T,13,,,2,1,MY8D56EYJC2VCPK29OVC53WWKH,2,2,1,2,3,5,4,BQR8ONT9Y4TNIK85XI84T5CQ2QHTLL88ERYD25J3,6ZB1H7CZ0FCHNZO24H8W9AHN6JZOR5OE364MRXPTA7WWJJ...,QE7MO6AKPAK5GZKVAGOMW798KCF7J,41,,,,,9UG6AUX6UPYO0A5W5L7MKTFIML7AO6O0ABV2AS7PX8XJZV...,65IZC38JED1MG0Y,Q4NUDRS19CF6S75NHTL55A4Z0M54LBA28AOZ1HE6TSOKWJ...,1,2,2,3,1,3,10,4,NA,0,NA,3,1,675,764,7,,4,,10,,,,,NA,NA,NA,NA,NA,11.11,NA,NA,98,NA,23,1,2,1,1,315777,3,5,3,NA,2,1,NA,4,5,,2,,J14GN1KT40J1NGIW35RONS5KWZN9Y3PLXP9SPJ1C7K8EJM...,9,8,,9,,,1,1,1,1
7,2,08ZQTJA357VRO2G0ZXGI,400RJ7FO656CJWHD8V9G3E,NA,4,4,2,2,3,287469,6,20181010,1234 Hocus Potato Way,Tatertown,WA,57630,47157,47157007900,4,,,,,M,2,1,,,,4ZOGFY57GSX0UC8X9V45D7GP8H4XD37G6HVYNZZ1AK9AW,3,1,7,,,,,8G2T12ARKQH0IKQE5UN9QRRY6VAPS1G7IFM0EK5GBFOFGE...,HVE4RX9KMF6K8F8KF1X24INN8ZW7OZ5JNKEEKVI7WNQ20Y...,67E3HNL818H2F9ZQ25P600I93EVE,7,,,,,7B20GC3HV3,REG8PK2FVDQ3SSQ,H46Q331AS6KB26F53Y97COPDJSDUJ4K6XO72VIL4BLKLXN...,3,3,3,1,3,2,53,81,147,0,NA,3,2,8888,8888,9,,9,,10,,,,,NA,NA,NA,NA,NA,NA,NA,NA,NA,145,21,2,2,2,1,NA,3,1,3,NA,3,3,NA,6,,,,,,17,,,,,,2,1,2,1
8,2,08ZQTJA357VRO2G0ZXGI,W5BV6YCT7D06B9RXF1O2OR,20180915,3,1,2,2,3,88165,4,20181110,1234 Hocus Potato Way,Tatertown,CO,29348,30063,30063001800,2,,13,,,XOMYEQRCVI54Q6UIJM,14,,1,,,ZXEJRWO3XZ58AEKIJ3JOM886Z,2,2,3,5,2,1,4,8SRT8O9OMS4EGLTUIPRZLQPXDXJ1RFTF6QEB1EBL2G3YZG...,JZTJGAYNN376LPJZG8F3C70IFLKIJYA1UQI7WRJLNRCYDI...,5SZ1K6KOE3I9SV3ZSYZO1GMJFXM5D3CX4H9JQIM2QOOCSS...,2,,,,,B4F0B26X5ZLBCX0MT10Q36ZETEWZR2P86B1ONLK4I1M5JX...,2R5X172UZD4X35FFLGTGJDYVGQ9F7E7GU8C,HW4HE2C2FG7A3H3BZXT2G3OQPJQX3HDTCU9HGU733479H1...,1,1,2,2,1,2,44,7,NA,0,NA,3,1,8888,8888,9,,9,,10,,,,,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,16,1,2,2,1,NA,3,5,17,17,1,2,NA,5,,,4,,IXNZI0FN9ZTDK0YDTSW8TBIPTSJBYLZRQ2S8L0Q7F011UU...,9,,,10,,,1,2,2,1
14,2,08ZQTJA357VRO2G0ZXGI,KA9JMUHLD068G835KL6IOG,20180923,2,31,2,2,3,360143,3,20180923,1234 Hocus Potato Way,Tatertown,DE,77025,55133,55133203802,14,,13,,,9C0NQRE8DR48W1OAG0A4G9BBHSW3BNBZK5HK2VS5N6R,4,,,,,YP6RK8,2,3,2,4,5,3,1,DE9ANNPW4SLLERZJP064UVOJ1S8MVGBBUP87BBU386DJRV...,O1BUZ9SHKZUGTV2617HBP2CPWG177VXM3HTV3NL,1EN5DZW,6,,,,,A8O8TMAMIKJ,E0Y5NTQ1NV30UZT0ASCULLTZ2TU7KQYAD9LDZZOLE3P1VU...,8K9N698YM3FRWGIHHCFB15WSFXL1O2CAALXS801GGTWF4L...,1,3,2,2,1,1,74,50,NA,0,NA,3,2,654,730,2,,8,KYIXWIYDQ88TRXZJU4UFXQPQC92X6XQWUMJD1V1HPQHKJW...,1,6,3,4,,NA,NA,NA,NA,NA,NA,NA,NA,105,NA,21,1,2,1,1,220862,3,5,3,NA,1,1,NA,1,,4,,4,,4,,11,,8,,1,2,2,1
15,2,08ZQTJA357VRO2G0ZXGI,PXN13ZNVDL9TIJWR1

In [152]:
file = "clean_file_100_rows.txt"
path = "../edits_files/"
clean_ts_df, clean_lar_df = utils.read_data_file(path=path, data_file=file)

In [119]:
test_ids = ["PI4DRJAWVZFPU98U02EYRE","PXN13ZNVDL9TIJWR1UWX8F","GT7H92U568QVXXR9I03X56","KLAY82I0HSO1S28EOIK1PA",
"RB2GHRGX76G2BY0SP9RW1Z", "C6DDWX48AG7OACRYOM5JTM", "AAMG10VXCBB0Y9YRIMTFU6", "3P639HSUNEULFD42X0FPI2", "N57IF6D4VQVK4Q23IXI8KM"]
clean_lar_df[clean_lar_df.uli.isin(test_ids)]

field,record_id,lei,uli,app_date,loan_type,loan_purpose,preapproval,const_method,occ_type,loan_amount,action_taken,action_date,street_address,city,state,zip_code,county,tract,app_eth_1,app_eth_2,app_eth_3,app_eth_4,app_eth_5,app_eth_free,co_app_eth_1,co_app_eth_2,co_app_eth_3,co_app_eth_4,co_app_eth_5,co_app_eth_free,app_eth_basis,co_app_eth_basis,app_race_1,app_race_2,app_race_3,app_race_4,app_race_5,app_race_native_text,app_race_asian_text,app_race_islander_text,co_app_race_1,co_app_race_2,co_app_race_3,co_app_race_4,co_app_race_5,co_app_race_native_text,co_app_race_asian_text,co_app_race_islander_text,app_race_basis,co_app_race_basis,app_sex,co_app_sex,app_sex_basis,co_app_sex_basis,app_age,co_app_age,income,purchaser_type,rate_spread,hoepa,lien,app_credit_score,co_app_credit_score,app_score_name,app_score_code_8,co_app_score_name,co_app_score_code_8,denial_1,denial_2,denial_3,denial_4,denial_code_9,loan_costs,points_fees,origination_fee,discount_points,lender_credits,interest_rate,prepayment_penalty,dti,cltv,loan_term,intro_rate,balloon,int_only_pmts,neg_amort,non_amort_features,property_value,manufactured_type,manufactured_interest,total_units,affordable_units,app_submission,initially_payable,mlo_id,aus_1,aus_2,aus_3,aus_4,aus_5,aus_code_5,aus_result_1,aus_result_2,aus_result_3,aus_result_4,aus_result_5,aus_code_16,reverse_mortgage,open_end_credit,business_purpose


In [115]:
clean_lar_df

field,record_id,lei,uli,app_date,loan_type,loan_purpose,preapproval,const_method,occ_type,loan_amount,action_taken,action_date,street_address,city,state,zip_code,county,tract,app_eth_1,app_eth_2,app_eth_3,app_eth_4,app_eth_5,app_eth_free,co_app_eth_1,co_app_eth_2,co_app_eth_3,co_app_eth_4,co_app_eth_5,co_app_eth_free,app_eth_basis,co_app_eth_basis,app_race_1,app_race_2,app_race_3,app_race_4,app_race_5,app_race_native_text,app_race_asian_text,app_race_islander_text,co_app_race_1,co_app_race_2,co_app_race_3,co_app_race_4,co_app_race_5,co_app_race_native_text,co_app_race_asian_text,co_app_race_islander_text,app_race_basis,co_app_race_basis,app_sex,co_app_sex,app_sex_basis,co_app_sex_basis,app_age,co_app_age,income,purchaser_type,rate_spread,hoepa,lien,app_credit_score,co_app_credit_score,app_score_name,app_score_code_8,co_app_score_name,co_app_score_code_8,denial_1,denial_2,denial_3,denial_4,denial_code_9,loan_costs,points_fees,origination_fee,discount_points,lender_credits,interest_rate,prepayment_penalty,dti,cltv,loan_term,intro_rate,balloon,int_only_pmts,neg_amort,non_amort_features,property_value,manufactured_type,manufactured_interest,total_units,affordable_units,app_submission,initially_payable,mlo_id,aus_1,aus_2,aus_3,aus_4,aus_5,aus_code_5,aus_result_1,aus_result_2,aus_result_3,aus_result_4,aus_result_5,aus_code_16,reverse_mortgage,open_end_credit,business_purpose
0,2,08ZQTJA357VRO2G0ZXGI,08ZQTJA357VRO2G0ZXGION5G4F8QESG553BMNIVSE9911,20180211,1,1,2,2,3,147842,5,20180718,1234 Hocus Potato Way,Tatertown,IL,79702,09009,09009175600,4,,,,,RNSN9XXSGSME1M3CYJEEID6DB7ZB7J3LXD4T1M8X2DCDK,2,1,,,,0F159LOGV8TTO2N7EUNFA,3,1,7,,,,,9WO4KCV3FP5U6QI09234OV4RAB,I3PSG,PDOX0F7X1LYKH,4,43,25,24,41,OJUDAVX1QXURZE11KL,VUS0WCP2IBH845IVQJ40H6MP69KCU3X01ENLPDQBBGWA50...,UYTYHMKP2XQXLEQID2U8L8AKCXDKFJHINP4PZSBNS10MQN...,3,2,6,1,2,2,82,5,NA,0,NA,3,2,8888,8888,9,,9,,10,,,,,NA,NA,NA,NA,NA,NA,NA,NA,NA,72,30,1,2,1,2,NA,3,5,1,NA,1,1,NA,1,1,1,5,3,0TDYGNMIH6YTV4MVH5DDTLO7KLK1UJLNKHZSITR94TY1YM...,4,1,2,11,1,,2,2,1
1,2,08ZQTJA357VRO2G0ZXGI,08ZQTJA357VRO2G0ZXGIL03AA1UT2H9RY0F6YCUZ5QX52,NA,3,2,2,2,1,307352,6,20181222,1234 Hocus Potato Way,Tatertown,PA,66738,17031,17031825600,2,,,,,PIQV98LKETR0SPDH18SZVH5WFWFY3C68MN10PUXI2H4GIU...,3,,,,,LJNM64K7FX6FMDIMB87DYI9JHEGYXGAPADCW4S3S1TEXLD...,1,3,6,,,,,ETEYIVU29IDKZ1BKQD9U5QK5FS,74K,N5TB2G3XV,8,,,,,IZOU5FE2,MO3JJ8NJL07XLLXTUKATSLJ0C4GINQRH0PYFJLETRHBGMM...,THRA01IDDSDQZQN,3,4,2,1,1,2,65,37,NA,1,NA,3,2,8888,8888,9,,9,,10,,,,,NA,NA,NA,NA,NA,1.01,NA,NA,NA,NA,21,2,2,1,2,489733,3,5,26,14,3,3,NA,6,,,,,,17,,,,,,1,2,2
2,2,08ZQTJA357VRO2G0ZXGI,08ZQTJA357VRO2G0ZXGIFXJIUB1HT8S81FO3TL86GKR28,NA,4,1,2,2,2,420620,6,20181217,1234 Hocus Potato Way,Tatertown,CA,81501,18097,18097357600,4,,,,,UZEXXRD9A6CF7HKNQ0F5JKKG3LXV7CPL6GP29IGHVRUDUA...,1,2,,,,4F5,3,1,26,4,,1,27,448FH,IA3ZNIEJSZ51Z72Q41LFVNUCQ6KF8LMP8D2NESZ3GM1T67...,ARC2QU8FCYP82T59J1OOUDW5AEFSPN0RH0EAOBEHY0C5PJ...,8,,,,,T6X0EOA,HUMOHHJC12V9QFSWFV3CT8C7JMOSIF8RT413ITDRCTGGMP...,O61DDDY7OAG5AI60T6QK9MBRGW4PNT3TC54,2,4,2,1,2,1,25,85,65,6,NA,2,2,8888,8888,9,,9,,10,,,,,NA,NA,NA,NA,NA,14.14,NA,NA,NA,184,34,2,1,1,1,600815,3,2,22,NA,3,3,NA,6,,,,,,17,,,,,,2,1,1
3,2,08ZQTJA357VRO2G0ZXGI,08ZQTJA357VRO2G0ZXGIFSR390BCHIGOY8VNRMIZCRM92,NA,3,4,2,2,2,306357,6,20180721,1234 Hocus Potato Way,Tatertown,TN,55589,04013,04013420209,11,,,13,,TC79W4C78KG29HLHWNNS16NIRAKF5ZAS4COCF7N,3,,,,,Y9FNCES1JU99TE9RK2CP3H16LF5N3QNGRUW3BC4H0O4JC1...,2,2,4,1,2,5,3,X00XRC8GK0161SRIIDRHUHFKUX,L07QXZVI,CB4BP2QA8ZLQZVR210GRJYTYPMMM50H1LQHCUUXHPVC54R...,41,,42,1,27,PQ9NEH9IO0CQFJFD7QKPQUWCVW1YQG1IKHW6T5GCJRUHJW...,MJEBXQLJQZQ28FTRYIM3ABIECEJFM9J6OB4VX3OC2BPBEU...,RFVGQ81DV0616HSLSBB1OKOCPIZ2CAQ40XS7P4D06B6H4S...,1,2,1,4,1,3,89,41,NA,0,NA,3,1,8888,8888,9,,9,,10,,,,,NA,NA,NA,NA,NA,NA,NA,NA,NA,72,8,2,1,2,1,NA,3,5,24,17,3,3,NA,6,,,,,,17,,,,,,2,1,2
4,2,08ZQTJA357VRO2G0ZXGI,KLB6E4CQIUDQG379R81UDQ,20180811,2,5,2,2,3,260765,2,20181204,1234 Hocus Potato Way,Tatertown,ID,37375,51041,51041100403,3,,,,,OCLG7DKYHGRFJLFFO5TR079U3T,13,

In [164]:
x = "W5BV6YCT7D06B9RXF1O2B"
x[:5]

'W5BV6'